In [2]:
!pip install tensorflow==1.14

     |████████████████████████████████| 109.2MB 89kB/s 
     |████████████████████████████████| 51kB 7.0MB/s 
     |████████████████████████████████| 3.2MB 48.4MB/s 
     |████████████████████████████████| 491kB 45.3MB/s 
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0


In [4]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np

bandit_arms=[-2,-0.2,0,0.2]
num_arms=len(bandit_arms)

def pullBandit(bandit):
    result = np.random.randn(1)
    if result > bandit :
        return 1
    else:    
        return -1

tf.reset_default_graph()

weights = tf.Variable(tf.ones([num_arms]))
output = tf.nn.softmax(weights)

reward_holder = tf.placeholder(shape=[1], dtype=tf.float32)
action_holder = tf.placeholder(shape=[1], dtype=tf.int32)

responsible_output = tf.slice(output,action_holder,[1])
loss=-(tf.log(responsible_output)*reward_holder)
optimizer = tf.train.AdamOptimizer(learning_rate=1e-3)
update = optimizer.minimize(loss)

total_episodes = 100
total_reward = np.zeros(num_arms)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    i = 0
    while i < total_episodes:
        actions = sess.run(output)
        a = np.random.choice(actions, p=actions)
        action = np.argmax(actions == a)

        reward = pullBandit(bandit_arms[action])

        _,resp,ww = sess.run([update, responsible_output, weights],feed_dict={reward_holder:[reward],action_holder:[action]})

        total_reward[action] += reward
        print("Runnig reward for the "+str(num_arms)+" arm of the bandit: "+str(total_reward))
        i+=1

print("\n The agent thinks arm" + str(np.argmax(ww)+1) + " is the most promising ....")
if np.argmax(ww) == np.argmax(-np.array(bandit_arms)):
    print(" ... and it was right !")
else:
    print(" ... and it was wrong .. ")

Runnig reward for the 4 arm of the bandit: [1. 0. 0. 0.]
Runnig reward for the 4 arm of the bandit: [2. 0. 0. 0.]
Runnig reward for the 4 arm of the bandit: [ 2. -1.  0.  0.]
Runnig reward for the 4 arm of the bandit: [ 3. -1.  0.  0.]
Runnig reward for the 4 arm of the bandit: [ 4. -1.  0.  0.]
Runnig reward for the 4 arm of the bandit: [4. 0. 0. 0.]
Runnig reward for the 4 arm of the bandit: [ 4. -1.  0.  0.]
Runnig reward for the 4 arm of the bandit: [ 5. -1.  0.  0.]
Runnig reward for the 4 arm of the bandit: [ 5. -1. -1.  0.]
Runnig reward for the 4 arm of the bandit: [ 5.  0. -1.  0.]
Runnig reward for the 4 arm of the bandit: [5. 0. 0. 0.]
Runnig reward for the 4 arm of the bandit: [ 5. -1.  0.  0.]
Runnig reward for the 4 arm of the bandit: [ 5. -1.  0. -1.]
Runnig reward for the 4 arm of the bandit: [ 6. -1.  0. -1.]
Runnig reward for the 4 arm of the bandit: [ 7. -1.  0. -1.]
Runnig reward for the 4 arm of the bandit: [ 7. -1.  1. -1.]
Runnig reward for the 4 arm of the bandi